# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

# for question 1
sum = 0
for sqrt_value in generator:
    #print(sqrt_value)
    sum = sum + sqrt_value
print(sum)


# For question 2
limit = 13
generator = square_root_generator(limit)

n=1
for sqrt_value in generator:
    if n==13:
        print(sqrt_value)
    n=n+1

8.382332347441762
3.605551275463989


## Answers to questions 1 and 2
### **Answer 1**: 8.38

### **Answer2**: 3.60

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


## For question 3

### First Load the first generator into duckdb

In [6]:
import dlt
import duckdb

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
							table_name="people_1",
							write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\juang_000\linuxprojects\DataTalksBootCamp\de\Homeworks\DTC-DE-course\hw-workshop-dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707931724.4108977 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707931724.4108977,Jatcc3gXqwgQAQ
1,2,Person_2,27,City_A,1707931724.4108977,QqnPPMZ9MgWUBA
2,3,Person_3,28,City_A,1707931724.4108977,78NVgl7pCBz1Ew
3,4,Person_4,29,City_A,1707931724.4108977,gfH4p26VYTftiQ
4,5,Person_5,30,City_A,1707931724.4108977,nKMxOzanmmGNZg


### Then Load the first generator into duckdb - 
In this case, based on the instruction of the question the write_disposition should be **append**

In [7]:
import dlt
import duckdb

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_2(),
							table_name="people_1",
							write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\juang_000\linuxprojects\DataTalksBootCamp\de\Homeworks\DTC-DE-course\hw-workshop-dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707931838.080528 is LOADED and contains no failed jobs


# Lastly, we check that everything was loaded to duckdb and sum all the ages

### **Answer 3**: 353

In [10]:
import duckdb

# show outcome
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

people_records = conn.sql("SELECT * FROM people_1").df()
display(people_records)

all_people_sum_age = conn.sql("SELECT sum(age) FROM people_1").df()
display(all_people_sum_age)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707931724.4108977,Jatcc3gXqwgQAQ,None
1,2,Person_2,27,City_A,1707931724.4108977,QqnPPMZ9MgWUBA,None
2,3,Person_3,28,City_A,1707931724.4108977,78NVgl7pCBz1Ew,None
3,4,Person_4,29,City_A,1707931724.4108977,gfH4p26VYTftiQ,None
4,5,Person_5,30,City_A,1707931724.4108977,nKMxOzanmmGNZg,None
5,3,Person_3,33,City_B,1707931838.080528,nU5L+q8HlASKhA,Job_3
6,4,Person_4,34,City_B,1707931838.080528,znuHMPM11MigCw,Job_4
7,5,Person_5,35,City_B,1707931838.080528,8j6w/e7XRInG6w,Job_5
8,6,Person_6,36,City_B,1707931838.080528,WPBRlJ2uewjGFA,Job_6
9,7,Person_7,37,City_B,1707931838.080528,azAwQzqkA05t+g,Job_7


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


## Loading the data of both generators on Duckdb

- In this case considering the write_disposition to **merge**
- Setting **ID** as the the primary key 

In [11]:
import dlt
import duckdb

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
							table_name="people_merged",
							write_disposition="merge",
                            primary_key="ID")
print(info)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_2(),
							table_name="people_merged",
							write_disposition="merge",
                            primary_key="ID")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.77 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\juang_000\linuxprojects\DataTalksBootCamp\de\Homeworks\DTC-DE-course\hw-workshop-dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707932415.7949843 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.61 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\juang_000\linuxprojects\DataTalksBootCamp\de\Homeworks\DTC-DE-course\hw-workshop-dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707932417.3196893 is LOADED and contains no failed jobs


## Now checking the data was loaded correctly and Calculating the sum of all the ages

In [14]:
import duckdb

# show outcome
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

people_records = conn.sql("SELECT * FROM people_merged").df()
display(people_records)

all_people_sum_age = conn.sql("SELECT sum(age) FROM people_merged").df()
display(all_people_sum_age)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1            │
│ people_merged       │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707932415.7949843,bsux+sIQ21HE9w,None
1,2,Person_2,27,City_A,1707932415.7949843,+jqDw4thzAbL+Q,None
2,5,Person_5,35,City_B,1707932417.3196893,+RZEnN6bt57ftQ,Job_5
3,3,Person_3,33,City_B,1707932417.3196893,IlQuwAREz4iVSQ,Job_3
4,6,Person_6,36,City_B,1707932417.3196893,wvTYp6/iR8+rhg,Job_6
5,8,Person_8,38,City_B,1707932417.3196893,oLXcN0OC+Jf6lQ,Job_8
6,4,Person_4,34,City_B,1707932417.3196893,pE2EP8OuikyzUA,Job_4
7,7,Person_7,37,City_B,1707932417.3196893,ylFXeuyTUOtIEQ,Job_7


,sum(age)
0,266.0


### **Answer 4**: 266